<a href="https://colab.research.google.com/github/Ramiro38/Modelo_de_IA_TM/blob/main/sprint4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CARGAR MODELO

In [ ]:
import os
# ponemos que queras sea compatible con las versiones 2.x
os.environ['TF_USE_LEGACY_KERAS'] = '1'

###subiendo modelo y etiquetas

In [ ]:
from tensorflow import keras

#Carga el modelo
model = keras.models.load_model("/content/drive/MyDrive/foticos-de-gato/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/foticos-de-gato/labels.txt", "r").readlines()

print(nombre_clases)

['0 Gato\n', '1 Perro\n']


#PONER IMAGEN

###se prepara la imagen antes de hacer las predicciones

In [ ]:
#importo la imagen y la abro en RGB
from PIL import Image, ImageOps
image = Image.open("/content/drive/MyDrive/foticos-de-gato/test/cat.9818.jpg").convert("RGB")

In [ ]:
# ajusto  el tamaño
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

In [ ]:
# lo paso a numpy
import numpy as np
image_array = np.asarray(image)

#MOSTRAR DATOS

In [ ]:
# hago que la array de imagen tenga valores entre -1 y 1
normalizada_imagen_array = (image_array.astype(np.float32) / 127.5) - 1

In [ ]:
# Creo un array para un lote de 1 imagen
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# asigno la imagen normalizada al lote
lote_imagenes[0] = normalizada_imagen_array

# hago las predicciones
resultados= model.predict(lote_imagenes)

indice = np.argmax(resultados[0])

1/1 [==============================] - 1s 1s/step


In [ ]:
# hago una lista de archivos con todas las imagenes de testeo
lista_archivos = os.listdir("/content/drive/MyDrive/foticos-de-gato/test")
print(lista_archivos)

['dog.9803.jpg', 'dog.9778.jpg', 'dog.9782.jpg', 'dog.9790.jpg', 'dog.9788.jpg', 'dog.9793.jpg', 'dog.9808.jpg', 'dog.9775.jpg', 'dog.9801.jpg', 'dog.9792.jpg', 'dog.9797.jpg', 'dog.9781.jpg', 'dog.9796.jpg', 'dog.9807.jpg', 'dog.9798.jpg', 'dog.9784.jpg', 'dog.9794.jpg', 'dog.9780.jpg', 'dog.9777.jpg', 'dog.9805.jpg', 'dog.9783.jpg', 'dog.9800.jpg', 'dog.9776.jpg', 'dog.9806.jpg', 'dog.9779.jpg', 'dog.9802.jpg', 'dog.9791.jpg', 'dog.9787.jpg', 'dog.9789.jpg', 'dog.9785.jpg', 'dog.9786.jpg', 'dog.9804.jpg', 'dog.9795.jpg', 'dog.9809.jpg', 'dog.9799.jpg', 'dog.9847.jpg', 'dog.9825.jpg', 'dog.9833.jpg', 'dog.981.jpg', 'dog.9854.jpg', 'dog.9820.jpg', 'dog.9840.jpg', 'dog.9850.jpg', 'dog.9827.jpg', 'dog.9859.jpg', 'dog.9835.jpg', 'dog.9845.jpg', 'dog.9814.jpg', 'dog.9852.jpg', 'dog.9828.jpg', 'dog.9817.jpg', 'dog.9810.jpg', 'dog.9857.jpg', 'dog.9849.jpg', 'dog.9841.jpg', 'dog.984.jpg', 'dog.9824.jpg', 'dog.9834.jpg', 'dog.9856.jpg', 'dog.9830.jpg', 'dog.9819.jpg', 'dog.9813.jpg', 'dog.9815

In [ ]:
# variables para el informe
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
predicciones_incorrectas = []

###Aqui se analiza cada imagen y la prediccion hecha se asignan a las variables del informe

In [ ]:
IMG_SIZE = (224, 224)
# funcion para predecir la imagen
def predecir_imagen(ruta_imagen):

    # Creo array del tamaño correcto
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # importo la imagen y la abro en RGB
    image = Image.open(ruta_imagen).convert("RGB")

    image = ImageOps.fit(image, IMG_SIZE, Image.Resampling.LANCZOS)

    # lo paso a numpy
    image_array = np.asarray(image)

    # hago que la array de imagen tenga valores entre -1 y 1
    normalized_image = (image_array.astype(np.float32) / 127.5) - 1

    data[0] = normalized_image

    # hago las predicciones
    pred = model.predict(data)[0]

    # Busco clase con mayor probabilidad
    idx_max = np.argmax(pred)
    etiqueta_predicha = nombre_clases[idx_max]
    probabilidad = float(pred[idx_max])

    return etiqueta_predicha, probabilidad


for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperaqda = "gato"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "perro"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/foticos-de-gato/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)



1/1 [==============================] - 0s 36ms/step


#INFORME

In [ ]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1

#imprimir el informe

informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 801
Aciertos: 0
Precisión: 0.00%
Probabilidad media de aciertos: 0.00%

Predicciones incorrectas:
Archivo: dog.9803.jpg, Predicción: 1 Perro
, Probabilidad: 86.49%
Archivo: dog.9778.jpg, Predicción: 1 Perro
, Probabilidad: 100.00%
Archivo: dog.9782.jpg, Predicción: 1 Perro
, Probabilidad: 97.10%
Archivo: dog.9790.jpg, Predicción: 1 Perro
, Probabilidad: 100.00%
Archivo: dog.9788.jpg, Predicción: 1 Perro
, Probabilidad: 99.78%
Archivo: dog.9793.jpg, Predicción: 1 Perro
, Probabilidad: 99.89%
Archivo: dog.9808.jpg, Predicción: 1 Perro
, Probabilidad: 67.25%
Archivo: dog.9775.jpg, Predicción: 1 Perro
, Probabilidad: 99.95%
Archivo: dog.9801.jpg, Predicción: 1 Perro
, Probabilidad: 93.32%
Archivo: dog.9792.jpg, Predicción: 1 Perro
, Probabilidad: 100.00%
Archivo: dog.9797.jpg, Predicción: 1 Perro
, Probabilidad: 99.99%
Archivo: dog.9781.jpg, Predicción: 1 Perro
, Probabilidad: 100.00%
Archivo: dog.9796.jpg, Predicción: 1 Perro
, Pro